In [4]:
import ee
import pandas as pd
from pathlib import Path
from datetime import datetime, timezone
import os
import re

from google.auth.transport.requests import AuthorizedSession

ee.Authenticate()  #  or !earthengine authenticate --auth_mode=gcloud
session = AuthorizedSession(ee.data.get_persistent_credentials())

##### Create empty image collections as target
Create empty image collection with earthengine CLI

Purge contents if delete or clean repopulation needed

List COGs in GCS folder

```
cd /data/gis/raster_base/Alaska/AKVegMap/disturbance

conda activate ee

earthengine create collection projects/akveg-map/assets/disturbance/most_recent_fire_year_v20250430

for i in `earthengine ls projects/akveg-map/assets/disturbance/most_recent_fire_year_v20250430`; do earthengine rm $i; done
earthengine ls projects/akveg-map/assets/disturbance/most_recent_fire_year_v20250430 | xargs -P 20 -n 1 earthengine rm

gsutil ls gs://akveg-data/disturbance/akcan/mrfy_*_v20250430_*.tif > mrfy_v20250430_on_gcs.txt
```

In [5]:
import json
# from urllib.parse import urlparse
import os
from pprint import pprint

In [6]:
# TODO Fix date extraction from filename to work for different filename formats
# Parts are zero-based
def load_gcs_cogs_to_collection(cogs, project_folder, collection, region_part, start_year_part, end_year_part, lfg_method_part, version_part):
    # Request body as a dictionary.
    for cog in cogs['tif']:
      fileOnly = os.path.split(cog)[1]
      # print(fileOnly)

      cogName = fileOnly[:-4]
      print(cogName)
      
      # parts = fileOnly.split('_')
      # Split on both underscore and period
      parts = re.split(r'[_\.]', fileOnly)
      print(parts)
      
      region = f"{parts[region_part]}"
      print(region)

      start_year = f"{parts[start_year_part]}"
      print(start_year)

      end_year = f"{parts[end_year_part]}"
      print(end_year)

      lfg_method = f"{parts[lfg_method_part]}"
      print(lfg_method)

      version = f"{parts[version_part]}"
      print(version)

      # start_dt = datetime(year, 1, 1)
      # start_dt = f"{start_dt:%Y-%m-%dT%H:%M:%SZ}"  # Format string for ISO 8601 + Z
      # end_dt = datetime(year + 1, 1, 1)
      # end_dt = f"{end_dt:%Y-%m-%dT%H:%M:%SZ}"  # Format string for ISO 8601 + Z
      # print(start_dt, end_dt)
        
      request = {
        'type': 'IMAGE',
        'gcs_location': {
          'uris': cog
        },
        'properties': {
            'region': region,
            'start_year': start_year,
            'end_year': end_year,
            'lfg_method': lfg_method,
            'version': version,
            #   'source': 'https://code.earthengine.google.com/067b10ee56537817756a3177a9138aee',
            # 'yyyymmdd_hhmmss': yyyymmdd_hhmmss
        },
        # 'startTime': start_dt,#'2023-01-01T00:00:00.000000000Z',
        # 'endTime': end_dt,#'2024-01-01T00:00:00.000000000Z',
      }

      # pprint(json.dumps(request))

      # A folder (or ImageCollection) name and the new asset name.
      asset_id = collection+'/'+cogName
      # print(project_folder)
      # print(asset_id)
        
      url = 'https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId={}'
      print(url)

      response = session.post(
        url = url.format(project_folder, asset_id),
        data = json.dumps(request)
      )

      pprint(json.loads(response.content))
    print('done')


## Run it

In [7]:
# earthengine ls projects/akveg-map/assets/disturbance/most_recent_fire_year_v20250430 | xargs -P 20 -n 1 earthengine rm
# gsutil ls gs://akveg-data/disturbance/akcan/mrfy_*_v20250430_*.tif > mrfy_v20250430_on_gcs.txt

project_folder = 'akveg-map'
collection = 'disturbance/most_recent_fire_year_v20250430'
tif_txt = '/data/gis/raster_base/Alaska/AKVegMap/disturbance/mrfy_v20250430_on_gcs.txt'
cogs = pd.read_csv(tif_txt, header=None,names=['tif'])

# def load_gcs_cogs_to_collection(cogs, project_folder, collection, region_part, start_year_part, end_year_part, lfg_method_part, version_part):
load_gcs_cogs_to_collection(cogs, project_folder, collection, 5, 2, 3, 1, 4)


mrfy_lfg1_2000_2023_v20250430_ak_3338_30m0000000000-0000000000
['mrfy', 'lfg1', '2000', '2023', 'v20250430', 'ak', '3338', '30m0000000000-0000000000', 'tif']
ak
2000
2023
lfg1
v20250430
https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId={}
{'bands': [{'dataType': {'precision': 'INT', 'range': {'max': 65535}},
            'grid': {'affineTransform': {'scaleX': 30,
                                         'scaleY': -30,
                                         'translateX': -2232345,
                                         'translateY': 2395455},
                     'crsCode': 'EPSG:3338',
                     'dimensions': {'height': 46592, 'width': 46592}},
            'id': 'fire_year',
            'missingData': {'values': [0]},
            'pyramidingPolicy': 'MEAN'}],
 'cloudStorageLocation': {'uris': ['gs://akveg-data/disturbance/akcan/mrfy_lfg1_2000_2023_v20250430_ak_3338_30m0000000000-0000000000.tif']},
 'geometry': {'coordinates': [[171.88934403735445, 68.3